In [48]:
import pandas as pd
import numpy as np
import spacy
import sec_edgar_downloader
import bs4
from spacy import matcher

In [2]:
dl = sec_edgar_downloader.Downloader(r"C:\Users\samuelabhi\Documents\pythonProjects\Market Intelligence\documentRepo")
dl.get("10-K", "MSFT", amount=1)
f = open(r"C:\Users\samuelabhi\Documents\pythonProjects\Market Intelligence\documentRepo\sec-edgar-filings\MSFT\10-K\0001564590-20-034944\filing-details.html", 'r', encoding = 'utf-8')
htmldoc = f.read()

In [3]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(htmldoc, 'html.parser')

In [4]:
import re

section = 'Risk Factors'

fullText = str(soup.get_text)

## Option 1 Identify boundary section IDs from Table of Contents 

startLink = soup.find_all('a', string = re.compile(section))[0]['href']

startSectionID = startLink[startLink.rfind('/')+2:]

endLink = [soup.find_all('a')[i+1]['href'] for i, link in enumerate(soup.find_all('a')) if link.string == section]

endSectionID = endLink[0][endLink[0].rfind('/')+2:]

startText = str(soup.find(id=startSectionID))
endText = str(soup.find(id=endSectionID))

startIndex = re.search(str(startText), str(fullText)).end()
endIndex = re.search(str(endText), str(fullText)).start()

extractedHtmlSection = fullText[startIndex:endIndex]

extractedSoup = BeautifulSoup(extractedHtmlSection, 'html.parser')

extractedText = extractedSoup.get_text()




64217

In [5]:
# A reliable way to get the risk headings but some of the section text is missing

extractedSubSections = [i.get_text() for i in extractedSoup.find_all('p', style = re.compile(r"font-weight:bold"))]

nlp = spacy.load("en_core_web_lg")

docsExtracted = list(nlp.pipe(extractedSubSections))

headers = [list(doc.sents)[0] for doc in docsExtracted]

In [94]:
# Going back to the extracted text to pick up sections for each header

doc = nlp(extractedText)

from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

headerPatterns = [nlp.make_doc(text.text) for text in headers]

matcher.add("10KSection", headerPatterns)

matches = matcher(doc)

a, b, prevEnd = matches[0]

sections = []

for _, start, end in matches[1:]:
    if (start - prevEnd) > 5:
        sections.append(doc[prevEnd:start])
    else:
        sections.append(doc[0:0])
    prevEnd = end
    
sections.append(doc[prevEnd:])

insights = zip(headers, sections)